# Zagadki

![image-2.png](attachment:image-2.png)

## Wstęp
Zagadki od wieków fascynują ludzi, pobudzając ich umysły do kreatywnego i logicznego myślenia. Od prostych łamigłówek po głębokie zagadki filozoficzne, stanowią one nie tylko formę rozrywki, ale również sztukę rozumienia języka i logicznego wnioskowania. W tym zadaniu będziesz rozwiązywał/a zagadki, polegające na odgadnięciu słowa na podstawie opisu. Wszystkie zagadki wymyślił ChatGPT (ale nie powiemy, która dokładnie wersja i w jaki sposób zachęcona), stąd niektóre mogą być trochę dziwne... Szacujemy, że ludzie są w stanie rozwiązać poprawnie trochę ponad 60% z nich. A jak dobry będzie Twój program?

## Zadanie
Napisz funckję `answer_riddle` która rozwiąże podaną na wejściu zagadkę. Rozwiązaniem jest zawsze jedno słowo. Przykładowe zagadki:

- **zagadka:** kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem <br>
  **odpowiedź:** pasażerka
- **zagadka:** emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu<br>
  **odpowiedź:** miłość


Naszym kryterium będzie `odwrócona średnia harmoniczna` ([Mean Reciprocal Rank](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)), która działa w następujący sposób: <br>
Jeżeli na zwróconej przez Twoją funkcję liście znajdzie się prawidłowa odpowiedź, otrzymasz wówczas punkty: dokładnie $\frac{1}{k}$ punktów, gdzie $k$ jest pozycją słowa na liście. W szczególności, jeżeli Twój program zgadnie słowo (czyli umieści je na pierwszej pozycji), to otrzymasz 1 punkt. Ostatecznym kryterium jest uśredniona liczba punktów ze wszystkich zagadek.

Powyższe kryterium jest zaimplementowane poniżej przez nas.

## Ograniczenia
- Twoje finalne rozwiązanie będzie testowane w środowisku **bez** GPU.
- Twoja funkcja powinna działać na tyle szybko, aby program był w stanie udzielić odpowiedzi na 100 zagadek w maksymalnie 2 minuty bez użycia GPU.

## Dane
Dane dostępne dla Ciebie w tym zadaniu to:
* `zagadki_do_testow_clean.txt` - około 2000 przykładowych zagadek
* `plwiktionary_definitions_clean.txt` -  plik z definicjami słów wziętymi z [pl.wiktionary.org](https://pl.wiktionary.org/wiki/pl). Z wszystkich definicji z pl.wiktionary.org wzięliśmy definicje 8094 najbardziej popularnych rzeczowników (częstości liczone zgodnie z korpusem https://2018.poleval.pl/index.php/tasks#task3). Uwaga: poprawne rozwiązanie każdej zagadki znajduje się w tym pliku!

* `superbazy_clean.txt` - formy bazowe polskich słów, przygotowane na podstawie projektu https://github.com/morfologik/polimorfologik

* Wektory osadzeń słów bazowych, wytrenowane modelem Word2Vec z biblioteki Gensim, na korpusie PolEval 2018 Task3

## Uwagi i wskazówki
- Dla każdej zagadki, Twoja funkcja powinna zwrócić listę słów (co najwyżej 20), w kolejności od najbardziej (wg Twojego programu) prawdopodobnej odpowiedzi na zagadkę, do najmniej.
- Twoje rozwiazanie bedzie testowane bez dostepu do internetu

## Pliki Zgłoszeniowe
Tylko ten notebook.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` będziesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających.

# Kod startowy

In [1]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################
FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

In [2]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
import nltk
from nltk.tokenize import word_tokenize as tokenize
from collections import defaultdict as dd
import math
from gensim.models import Word2Vec
import gdown
import random
import os
from tqdm import tqdm

## Ładowanie danych

In [3]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
path_to_data = 'data/'

bases = {}
# Dictionary mapping words to their base words
all_word_definitions = dd(list)
# Dictionary containing all base words inverse document frequency
base_idf = dd(int)

In [4]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def get_word_base(word):
    global bases
    word = word.lower()
    ret = bases.get(word)

    if ret:
        ret = ret.split("_")
        return ret[0]
        # return ret
    return word

In [5]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    if not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.syn1neg.npy") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.wv.vectors.npy"):
            gdown.download_folder(url="https://drive.google.com/drive/folders/1P72og_ORfL3Ojf27n-g06DT0ENduPy8C?usp=sharing", output=f"./{path_to_data}")
    nltk.download('punkt')

Retrieving folder contents


Processing file 1dCYKzdg6TihyckNM9Zxr1g9lWT1BzDXI plwiktionary_definitions_clean.txt
Processing file 1q6Ki5Y66gugM30oFcCtJj7ocMJymskSk superbazy_clean.txt
Processing file 1GZPNIR16bxFnzvbIYDLacn8TcKwiGnIh w2v_polish_lemmas.model
Processing file 1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC w2v_polish_lemmas.model.syn1neg.npy
Processing file 1RY0Ftfx_-nPUbddYCvHq9p8yCcXbUrYS w2v_polish_lemmas.model.wv.vectors.npy
Processing file 18wrF9VyESTvIT9Z_TqUd_2wV4lP2As3t zagadki_do_testow_clean.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1dCYKzdg6TihyckNM9Zxr1g9lWT1BzDXI
To: /content/data/zagadki/plwiktionary_definitions_clean.txt
100%|██████████| 1.59M/1.59M [00:00<00:00, 92.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1q6Ki5Y66gugM30oFcCtJj7ocMJymskSk
To: /content/data/zagadki/superbazy_clean.txt
100%|██████████| 43.7M/43.7M [00:00<00:00, 44.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GZPNIR16bxFnzvbIYDLacn8TcKwiGnIh
To: /content/data/zagadki/w2v_polish_lemmas.model
100%|██████████| 40.8M/40.8M [00:01<00:00, 36.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC
From (redirected): https://drive.google.com/uc?id=1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC&confirm=t&uuid=273cc6a6-09a0-4c54-892d-93b3c3293b50
To: /content/data/zagadki/w2v_polish_lemmas.model.syn1neg.npy
100%|██████████| 958M/958M

In [6]:
files = os.listdir(path_to_data + "zagadki")
print("Pobrane pliki:")
for file in files:
    print(file)



Pobrane pliki:
plwiktionary_definitions_clean.txt
w2v_polish_lemmas.model.wv.vectors.npy
superbazy_clean.txt
w2v_polish_lemmas.model
w2v_polish_lemmas.model.syn1neg.npy
zagadki_do_testow_clean.txt


In [7]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
for x in open(f'{path_to_data}/zagadki/superbazy_clean.txt'):
    word,base = x.lower().split()
    bases[word] = base

In [8]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
model = Word2Vec.load(f'{path_to_data}/zagadki/w2v_polish_lemmas.model')

In [9]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
for x in open(f'{path_to_data}/zagadki/plwiktionary_definitions_clean.txt'):
    word, definition = x.split('###')
    L = word.split()
    if len(L) == 1:
        word = L[0]

        definition = set(tokenize(definition.lower()))
        all_word_definitions[word].append(definition)
        for word in set(definition):
            base_idf[get_word_base(word)] += 1


for base in base_idf:
    base_idf[base] = -math.log(base_idf[base] / len(all_word_definitions))

In [10]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
answers = []
queries = []

with open(f'{path_to_data}/zagadki/zagadki_do_testow_clean.txt') as file:
    for line in file:
        line = line.replace(';;', '').split()
        answers.append(line[0])
        queries.append(tokenize(' '.join(line[1:])))

## Kod z kryteriami oceniającymi

In [11]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def mean_reciprocal_rank(real_answers, computed_answers, K=20):
    positions = []

    for real_answer, computed_answer in zip(real_answers, computed_answers):
        if real_answer in computed_answer[:K]:
            pos = computed_answer.index(real_answer) + 1
            positions.append(1/pos)

    mrr = sum(positions) / len(real_answers)
    print ('Mean Reciprocal Rank =', mrr)

    return mrr

# Twoje rozwiązanie

To jest jedyna sekcja, w której musisz coś zrobić.

In [12]:
def clear_riddle(riddle):
    cleared_riddle = np.array([])

    # for word in riddle:
    #     cleared_riddle = np.append(cleared_riddle, word.replace("'", " "). replace('"'," "))

    # list_to_remove = [",",".","?","-", "czy", "i", "{", "}"]
    list_to_remove = [",",".","?", "!", "#", "%", "'", "''", '"', '""',"-", "czy", "i", "{", "}", ";", "w", "lub", "(", ")", "wikipedia", "z", "do", "na", ".", ":", "_", "-","/","@", "^", "&", "*","+"]
    for el in list_to_remove:
        riddle.discard(el)

    for word in riddle:
      cleared_riddle = np.append(cleared_riddle, get_word_base(word))

    return cleared_riddle

def vectorize_sentence(riddle):
    global model

    words = np.array([])
    weights = np.array([])

    for word in riddle:
        if (word in model.wv):
            words = np.append(words, word)
            if(word in base_idf):
                if(base_idf[word]<0):
                    weights = np.append(weights, 0)
                else:
                    weights = np.append(weights, base_idf[word])
            else:
                weights = np.append(weights, 34)
    if (len(words)>0):
        word_vectors = np.array([model.wv[word] for word in words])

        average_vector = np.average(word_vectors, axis=0, weights=weights)

        return average_vector
    return np.nan

def closest_words_to_vectorized_string(riddle,vectorized_string, topn=20):
    global model

    closest_words = []
    for word, definitions in all_word_definitions.items():
        if(get_word_base(word) in model.wv and get_word_base(word) not in riddle):
            word_vector = model.wv[get_word_base(word)]
            similarity = np.dot(vectorized_string, word_vector) / (np.linalg.norm(vectorized_string) * np.linalg.norm(word_vector))
            closest_words.append((word, similarity))
            if(word in vectorized_definitions):
                for word_vector in vectorized_definitions[word]:
                    similarity = np.dot(vectorized_string, word_vector) / (np.linalg.norm(vectorized_string) * np.linalg.norm(word_vector))
                    closest_words.append((word, similarity))

    closest_words.sort(key=lambda x: x[1], reverse=True)

    return closest_words[:topn]

In [13]:
# base_idf2 = base_idf.copy();
# list(filter(lambda x: len(x[0])<2,sorted(base_idf2.items(), key = lambda x: x[1])))

In [20]:
def vectorize_words(riddle):
    global model

    words = np.array([])
    weights = np.array([])

    for word in riddle:
        if (word in model.wv):
            words = np.append(words, word)
            if(word in base_idf):
                if(base_idf[word]<0):
                    weights = np.append(weights, 0)
                else:
                    weights = np.append(weights, base_idf[word])
            else:
                weights = np.append(weights, 34)
    if (len(words)>0):
        word_vectors = torch.tensor(np.array([model.wv[word] for word in words]))

        # average_vector = np.average(word_vectors, axis=0, weights=weights)

        return word_vectors
    return torch.tensor([])

In [21]:
import torch
import numpy as np

class VectorizationModel(torch.nn.Module):
    def __init__(self):
        super(VectorizationModel, self).__init__()
        self.embedding_dim = 200
        self.hidden_dim = 256
        self.num_layers = 2

        self.bilstm = torch.nn.LSTM(self.embedding_dim, self.hidden_dim, self.num_layers, bidirectional=True, batch_first=True)
        self.fc = torch.nn.Linear(self.hidden_dim*self.num_layers, 200)


    def forward(self, x):
        x, _ = self.bilstm(x)
        x = self.fc(x)[-1,:]

        return x.squeeze(-1)

In [16]:
print(len(all_word_definitions.items()))

8085


In [22]:
def train_model(model_t, data):
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model_t.parameters(), lr=0.001)

    i = 0
    for word, definitions in data.items():
        if i > 300:
            break
        print(i)
        i += 1
        for single_definition in definitions:
            vectorized = vectorize_words(single_definition)

            if vectorized.size()[0] != 0 and word in model.wv:
                output = model_t(vectorize_words(single_definition))

                target = torch.tensor(model.wv[word])
                loss = criterion(output, target)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

    return model_t

vec_model = VectorizationModel()
vec_model = train_model(vec_model, all_word_definitions)

In [24]:
import numpy as np
vectorized_definitions = {}

for word, definitions in all_word_definitions.items():
            for single_definition in definitions:
                if (len(single_definition) != 0):
                    word_vector = vectorize_sentence(clear_riddle(single_definition))
                    if (word_vector is not np.nan):
                        if (word in vectorized_definitions):
                            vectorized_definitions[word] = np.append(vectorized_definitions[word],[word_vector], axis=0)
                        else:
                            vectorized_definitions[word] = np.array([word_vector])

In [25]:
max_len = 0

for word, definitions in all_word_definitions.items():
    for definition in definitions:
        if len(definition) > max_len:
            max_len = len(definition)

print(max_len)

35


In [27]:
# i = 0
# for word in vectorized_definitions:
#     if i>10:
#       break
#     print(word)
#     print(vectorized_definitions[word].shape)
#     i+=1

In [31]:
# ///////////////////////////////////////\
#
# PAMIĘTAJ O ZMIANIE W get_word_base()
# I o zmianie w definiton.replace("'"," ")
#
# //////////////////////////////////////

import numpy as np

def get_word_base_my(word):
    return get_word_base(word)

def answer_riddle(riddle, K):
    global model

    riddle = clear_riddle(riddle)


    # average_vector = vectorize_sentence(riddle)
    with torch.no_grad():
        vec_model.eval()
        average_vector = vec_model(vectorize_words(riddle))

        similar_words_with_probabilities = closest_words_to_vectorized_string(riddle,average_vector, K)

        # similar_words_with_probabilities = model.wv.similar_by_vector(average_vector, topn=K)
        similar_words = [word[0] for word in similar_words_with_probabilities]

    return similar_words

In [32]:
answer_riddle(set(tokenize("dawno temu za górami za lasami")),20)

['numer',
 'model',
 'klasa',
 'film',
 'instancja',
 'model',
 'standard',
 'adnotacja',
 'detal',
 'mechanika',
 'czerwień',
 'skrzydło',
 'plazma',
 'czwórka',
 'tabletka',
 'model',
 'aspekt',
 'stan',
 'stanik',
 'staw']

# Ewaluacja

Poniższy kod będzie służył ewaluacji rozwiązania. Po wysłaniu rozwiązania do nas zostanie wykonana funkcja `evaluate_algorithm(score_function, queries, answers, K)`, t.j. prawie identyczny kod jak niżej będzie się uruchamiał na katalogu danych `test_data` dostępnym tylko dla sprawdzających zadania.

Upewnij się przed wysłaniem, że cały notebook wykonuje się od początku do końca bez błędów i bez ingerencji użytkownika po wykonaniu polecenia `Run All`.

In [35]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def evaluate_algorithm(score_function, queries, answers, K):
    computed_answers = []
    i=0
    for query in tqdm(queries, desc="queries answered"):
        computed_answers.append(score_function(set(query), K=K))
        # print(queries[i])
        print(answers[i])
        print(computed_answers[i])
        # print("///////////////////////////////////")
        i+=1
    score = mean_reciprocal_rank(answers, computed_answers, K=K)

    return score

In [37]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    PART_OF_DATA = 100
    START = 0
    K = 20
    valid_queries = queries[START:PART_OF_DATA]
    valid_answers = answers[START:PART_OF_DATA]

    # for i in range(PART_OF_DATA):
    #   print(valid_queries[i])
    #   print(valid_answers[i])
    #   print(all_word_definitions[valid_answers[i]])

    score = evaluate_algorithm(answer_riddle, valid_queries, valid_answers, K=K)
    print(f"Score: {score}")

queries answered:   1%|          | 1/100 [00:01<01:56,  1.18s/it]

manuskrypt
['numer', 'adnotacja', 'klasa', 'kolekcja', 'model', 'marka', 'skrzydło', 'instancja', 'marka', 'standard', 'tabletka', 'czerwień', 'mechanika', 'interferon', 'model', 'komoda', 'detal', 'krąg', 'czwórka', 'film']


queries answered:   2%|▏         | 2/100 [00:02<02:01,  1.24s/it]

wesołość
['numer', 'klasa', 'standard', 'marka', 'adnotacja', 'model', 'marka', 'kolekcja', 'mechanika', 'stopa', 'model', 'komoda', 'czerwień', 'model', 'skrzydło', 'instancja', 'adidas', 'krąg', 'żyła', 'tabletka']


queries answered:   3%|▎         | 3/100 [00:03<01:55,  1.19s/it]

...

nurkowanie
['numer', 'adnotacja', 'klasa', 'kolekcja', 'marka', 'skrzydło', 'marka', 'tabletka', 'czerwień', 'komoda', 'krąg', 'instancja', 'model', 'interferon', 'mechanika', 'standard', 'model', 'czwórka', 'chwila', 'żyła']


queries answered:   7%|▋         | 7/100 [00:08<01:49,  1.18s/it]

nonsens
['numer', 'klasa', 'adnotacja', 'standard', 'marka', 'kolekcja', 'marka', 'komoda', 'czerwień', 'skrzydło', 'mechanika', 'krąg', 'model', 'model', 'instancja', 'starter', 'żyła', 'pik', 'stopa', 'czwórka']


queries answered:   7%|▋         | 7/100 [00:09<02:09,  1.39s/it]


KeyboardInterrupt: 

# ZRÓB KOPIĘ

Zrób kopięęęęęęęęęęęęęęęęę